# Models with immunity

## SIR-O - a modified SIR with explicit confirmed cases
This model is modified [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model): 
- **S** is *Susceptible*
- **I** is *Infectious*
- **R** is *Recovered* (and *diseased*). The model expects that this pool becomes 100% immune.
- **O** is *cOnfirmed* individuals, both hospitalized and quorantined at home with a positive test

The model utilises three parameters: $\alpha$,$\beta$ and $k$:
- $\alpha$ models the speed of infection spread,
- $\beta$ represents the speed of recovery (and deaths)
- $k$ is the reveal factor, how many infectious people are confirmed as positive.

The differential equations:<br>
$\frac{dS}{dt} = -\alpha S I$,<br>
$\frac{dI}{dt} = \alpha S I - \beta I$, <br>
$\frac{dR}{dt} = \beta I$,<br>
$\frac{dO}{dt} = k I$.


# Models without immunity

## SIRS-O - a SIR model without antigen and with explicit confirmed cases
This model is modified [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model): 
- **S** is *Succeptible*
- **I** is *Infectious*
- **R** is *Recovered* (and *diseased*). While in **R** state people are immune.
- **O** is *cOnfirmed* individuals, both hospitalized and quorantined at home with a positive test

The major feature of this model is conversion of some or recovered back into succeptible.

The model utilises three parameters: $\alpha$,$\beta$ and $k$:
- $\alpha$ models the speed of infection spread,
- $\beta$ represents the speed of recovery (and deaths),
- $r_s$ is the ratio of recovered people who are susceptible again,
- $k$ is the reveal factor, how many infectious people are confirmed as positive.

The differential equations:
$\frac{dS}{dt} = -\alpha S I + r_s R$,<br>
$\frac{dI}{dt} = \alpha S I - \beta I$, <br>
$\frac{dR}{dt} = \beta I - r_s R$,<br>
$\frac{dO}{dt} = k I$.

## SIRC-O - a SIR model with carriers and with explicit confirmed cases
This model is modified [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model): 
- **S** is *Succeptible*
- **I** is *Infectious*, i.e. with syptoms of the disease.
- **C** is *Carrier*, i.e. infectious, but without symptoms or actual illness.
- **R** is *Recovered* (and *diseased*). While in **R** state people are immune.
- **O** is *cOnfirmed* individuals, both hospitalized and quorantined at home with a positive test

The major feature of this model is having people carry the virus and infect others without actually being ill.

The model utilises three parameters: $\alpha$,$\beta$ and $k$:
- $\alpha$ models the speed of infection spread,
- $\beta$ represents the speed of recovery (and deaths),
- $c_s$ models efficiency of infection transmission by carriers compared to infectious.
- $i_r$ is the ratio of infectious, who recovered (became immune), and $1 - i_r$ become carriers.
- $c_r$ represents the speed of recovery of carriers.
- $k$ is the reveal factor, how many infectious people are confirmed as positive.

The differential equations:
$\frac{dS}{dt} = -\alpha S I - c_s \alpha S C$,<br>
$\frac{dI}{dt} = \alpha S I + c_s \alpha S C - \beta I$, <br>
$\frac{dR}{dt} = \beta i_r I$,<br>
$\frac{dC}{dt} = \beta (1 - i_r) I$,<br>
$\frac{dO}{dt} = k (I + C)$.



In [ ]:
country  = 'Russia'
country2 = 'Russian Federation'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit
from scipy.integrate import odeint
from datetime import datetime

%matplotlib inline

In [ ]:
print("Computed on: ", datetime.now().strftime(format='%y-%m-%d %T'))

In [ ]:
import corona

In [ ]:
info = corona.CovidModel(country, country2)
print("Country: ", info.country)
print("Population: ", info.population)
print("Number of revealed cases: ", info.data[0][-1])
print("Number of deaths: ", info.data[1][-1])

In [ ]:
delta = info.data[:,1:] - info.data[:,:-1]

days = np.arange(np.shape(delta)[1])
fig, ax1 = plt.subplots()

ax1.bar(days, delta[0], width=1.0)
ax1.set_ylabel('Cases per day')

ax2 = ax1.twinx()
ax2.plot(days, info.data[0][1:], 'o', color='red')
ax2.set_ylabel('Total number of cases')
fig.tight_layout()

plt.title(info.country + ": COVID19 stats");

# Models without immunity

## Some of recovered become susceptible again

In [ ]:
test_sirs = corona.CovidModel(country, country2, model='sir_so_r0')
test_sirs.run(300)

## Part of the population are symptom-less infectious carriers
Recovered posess immunity

In [ ]:
test_sirc = corona.CovidModel(country, country2, model='sir_co')
test_sirc.run(300)

## Part of the population are symptom-less infectious carriers, and recovered loose the immunity

In [ ]:
test_sir_sco = corona.CovidModel(country, country2, model='sir_sco')
test_sir_sco.run(300)


## Experimental model with carriers and no immunity
The major change from the previous model - difference in revealing efficiency for infectious and carriers

In [ ]:
test_sir_sco_kc = corona.CovidModel(country, country2, model='sir_sco_kc')
test_sir_sco_kc.run(300)

## Model with incubation period and no immunity (SEIRS)

In [ ]:
test_seir_so = corona.CovidModel(country, model='seir_so')
test_seir_so.run(300)

# Models with immunity

## SIR - the basic epidemiology model

In [ ]:
test_sir = corona.CovidModel(country, country2, model='sir_o_r0')

test_sir.run(500, figsize=(10, 6))

## Model with exposed (incubation period)

In [ ]:
test_seir = corona.CovidModel(country, model='seir_o')

test_seir.run(300)